In [1]:
properties_2016 = pd.read_csv('../../data/properties_2016.csv')

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [31]:
train_df_2016 = pd.read_csv('../../data/train_2016_v2.csv')
train_2016 = pd.merge(train_df_2016, properties_2016, how ='left', on ='parcelid')

### 세금과 관련된 컬럼 fill
- structuretaxvaluedollarcnt = taxvaluedollarcnt - landtaxvaluedollarcnt
- taxamount : taxamount와 taxvaluedollarcnt의 비율을 구한다음 평균값을 구해서 taxvaluedollarcnt에 곱해준 값을 채움
- taxdelinquencyflag : Y값은 1로 대체 NaN 값은 0으로 대체

In [32]:
train_2016.loc[np.isnan(train_2016["taxvaluedollarcnt"]), "taxvaluedollarcnt"] = np.mean(train_2016["taxvaluedollarcnt"].notnull())
train_2016.loc[np.isnan(train_2016["landtaxvaluedollarcnt"]), "landtaxvaluedollarcnt"] = np.mean(train_2016["landtaxvaluedollarcnt"].notnull())
train_2016["structuretaxvaluedollarcnt"] = train_2016["taxvaluedollarcnt"] - train_2016["landtaxvaluedollarcnt"]
train_2016.loc[train_2016["taxdelinquencyflag"] == 'Y', "taxdelinquencyflag"] = 1.0
train_2016.loc[train_2016["taxdelinquencyflag"].isnull(), "taxdelinquencyflag"] = 0.0   

### 층과 관련된 컬럼 Fill
- numberofstories : structuretaxvaluedollarcnt가 0이면 0, 나머지는 1.0으로 채워줌, 최빈값
- 건물세가 없고 층이 Nan값이면 0으로 채움
- 1층 면적이 있고 층이 Nan값이면 1로 채워줌

In [33]:
train_2016.loc[train_2016["numberofstories"].isnull() & train_2016["structuretaxvaluedollarcnt"]==0.0 ,"numberofstories"] = 0.0
train_2016.loc[(train_2016["numberofstories"] == 0.0) & train_2016["finishedfloor1squarefeet"].notnull(), "numberofstories"] = 1.0
train_2016["numberofstories"].fillna(value = 1.0, inplace = True)
#storytypeid: 0으로 채워줌(유무)
train_2016["storytypeid"].fillna(value = 0, inplace = True)
train_2016["basementsqft"].fillna(value = 0, inplace = True)

### 면적과 관련된 컬럼 fill

In [34]:
# 총면적이 1층 면적보다 작은경우 1층 면적을 총면적에 대입
train_2016.loc[train_2016["calculatedfinishedsquarefeet"].notnull() < train_2016["finishedsquarefeet50"].notnull(), "calculatedfinishedsquarefeet"] = train_2016["finishedsquarefeet50"]
# 층수가 없는 경우 총면적의 NaN은 0
train_2016.loc[(train_2016["numberofstories"] == 0.0) & train_2016["calculatedfinishedsquarefeet"].isnull(), "calculatedfinishedsquarefeet"] = 0.0
# 건물이 있지만 총면적이 0인 경우 총면적의 평균을 대입
train_2016.loc[(train_2016["numberofstories"] != 0.0) & train_2016["calculatedfinishedsquarefeet"].isnull(),"calculatedfinishedsquarefeet"] = np.mean(train_2016["calculatedfinishedsquarefeet"].notnull())
# 1층의 면적을 나타내는 finishedfloor1squarefeet, finishedsquarefeet50의 Nan값은 calculatedfinishedsquarefeet의 값으로 대체(대부분 비슷하지만 조금 차이가남)
train_2016.loc[train_2016["finishedfloor1squarefeet"].isnull(), "finishedfloor1squarefeet"] = train_2016["calculatedfinishedsquarefeet"]
train_2016.loc[train_2016["finishedsquarefeet50"].isnull(), "finishedsquarefeet50"] = train_2016["calculatedfinishedsquarefeet"]
train_2016.loc[train_2016["finishedsquarefeet6"].isnull(), "finishedsquarefeet6"] = train_2016["calculatedfinishedsquarefeet"]
# 총면적과 거주 공간외 공간의 면적이 같은 경우 거주 공간의 면적을 0으로 넣어줌
train_2016.loc[train_2016["calculatedfinishedsquarefeet"].notnull() == train_2016["finishedsquarefeet13"].notnull(), "finishedsquarefeet12"] = 0.0
# finishedsquarefeet12의 NaN값은 calculatedfinishedsquarefeet와 일치 
train_2016.loc[train_2016["finishedsquarefeet12"].isnull(), "finishedsquarefeet12"] = train_2016["calculatedfinishedsquarefeet"]
train_2016["finishedsquarefeet13"].fillna(value = 0.0, inplace=True)
# finishedsquarefeet15 : calculatedfinishedsquarefeet의 값으로 채워줌
train_2016.loc[train_2016["finishedsquarefeet15"].isnull(), "finishedsquarefeet15"] = train_2016["calculatedfinishedsquarefeet"]
# lotsizesquarefeet의 값이  finishedsquarefeet50(1층면적) 보다 작은 경우 1층면적으로 대체
train_2016.loc[train_2016["finishedsquarefeet50"] > train_2016["lotsizesquarefeet"], "lotsizesquarefeet"] = train_2016["finishedsquarefeet50"]
# calculatedfinishedsquarefeet(총면적)이 있는데 lotsizesquarefeet이 NaN인 경우 1층의 값으로 대입
train_2016.loc[train_2016["lotsizesquarefeet"].isnull() & train_2016["calculatedfinishedsquarefeet"].notnull(), "lotsizesquarefeet"] = train_2016["finishedsquarefeet50"]

train_2016.loc[train_2016["yearbuilt"].isnull() & train_2016["buildingqualitytypeid"] == 7.0, "yearbuilt"] = 1950
train_2016.loc[train_2016["yearbuilt"].isnull() & train_2016["buildingqualitytypeid"] == 4.0, "yearbuilt"] = 1979
train_2016["yearbuilt"].fillna(value = 1955, inplace = True)

train_2016.loc[train_2016["unitcnt"].isnull() & (train_2016["calculatedfinishedsquarefeet"] == 0), "unitcnt"] = 0
train_2016["unitcnt"].fillna(value = 1, inplace = True)

### 건설 시기, 건물 구조물 등 컬럼 fill

In [35]:
train_2016["decktypeid"].fillna(value = 0.0, inplace = True)
train_2016.loc[train_2016["decktypeid"] == 66.0 , "decktypeid"] = 1
train_2016.loc[train_2016["buildingqualitytypeid"].isnull() & (train_2016["structuretaxvaluedollarcnt"] == 0), "buildingqualitytypeid"]=0
train_2016.loc[train_2016["buildingqualitytypeid"].isnull(), "buildingqualitytypeid"] = train_2016["buildingqualitytypeid"].mode()
train_2016.loc[train_2016["buildingclasstypeid"].notnull(), "buildingclasstypeid"] = 1
train_2016.loc[train_2016["buildingclasstypeid"].isnull(), "buildingclasstypeid"] = 0
train_2016["architecturalstyletypeid"].fillna(value=7, inplace=True)
train_2016['typeconstructiontypeid'].fillna(value=6, inplace=True)

In [36]:
type_properties_2016 = properties_2016.dtypes.reset_index()
type_properties_2016.columns = ["columns", "type"]
type_properties_2016

,columns,type
0,parcelid,int64
1,airconditioningtypeid,float64
2,architecturalstyletypeid,float64
3,basementsqft,float64
4,bathroomcnt,float64
5,bedroomcnt,float64
6,buildingclasstypeid,float64
7,buildingqualitytypeid,float64
8,calculatedbathnbr,float64
9,decktypeid,float64


In [38]:
model = sm.OLS.from_formula("logerror ~ C(numberofstories) +C(unitcnt) + scale(calculatedfinishedsquarefeet)"
                            "+scale(finishedsquarefeet13)+ scale(landtaxvaluedollarcnt) + scale(structuretaxvaluedollarcnt)"
                            "+scale(taxamount)+ scale(lotsizesquarefeet) + 0", train_2016)
print(model.fit().summary())

ValueError: zero-size array to reduction operation maximum which has no identity

In [ ]:
cols = ["logerror", "finishedfloor1squarefeet",  "calculatedfinishedsquarefeet", "finishedsquarefeet12", "finishedsquarefeet13", "finishedsquarefeet15", "finishedsquarefeet50", "finishedsquarefeet6", "lotsizesquarefeet"]
sns.pairplot(train_2016[cols])
plt.show()

In [ ]:
dfX = dmatrix(formula, dfX0, return_type="dataframe")
cmap = sns.light_palette("darkgray", as_cmap=True)
sns.heatmap(fill_merge_df_2016[cols].corr(), annot=True, cmap=cmap)
plt.show()